# SVM

Objective: Build a model to predict if a single compound is a "PPI modulator" or is not.

Data: ADME descriptors for 3 libraries.
    Libraries:
        AFRODB
        Biofacquim
        FDA
        PPI
        
        
    Endpoint: "PPI modulator" (Binary)
        1 -> PPI modulator
        0 -> Not PPI modulator
        
    Descriptors
        ADME descriptors:
            '#Aromatic heavy atoms'
            '#H-bond acceptors'
            '#H-bond donors'
            '#Heavy atoms'
             
Method: Support Vector Machine

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import os

from sklearn.preprocessing import label_binarize
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import roc_curve, auc

from Functions_SVM import test_compound,test_compound_real_category, svm_report, plot_roc

In [2]:
descriptors = ['HBA', 'HBD', 'RB', 'LogP', 'TPSA', 'MW', 'Heavy Atom', 'Ring Count', 'Fraction CSP3',]
#Falta generar los descriptores

In [3]:
#Difacquim computer root
root = "/home/barbara/Documents/DIFACQUIM/PPI_classifier/phase-1/Databases/"

In [4]:
class SVM:
    
    def __init__(self, input_file, target, descriptors, fraction):
        self.Data  = pd.read_csv(root+str(input_file))
        self.fraction = fraction
        #Muestreo
        self.Data = pd.DataFrame.sample(self.Data, frac=0.3, replace=True,  random_state=1992, axis=None) 
        print(self.Data.PPI.unique())
        print("Libraries are: ", self.Data.Library.unique())
        #print("PPI modulator: ", self.Data[target].unique())
        print("Total compounds ", self.Data.shape[0])
        self.descriptors = descriptors
        self.target = target
        
    def train_model(self, kernel):
        """
        fraction: float, indicates test set composition
        kernel: str, customize kernel (see scikit learn documentation)
        """
        y = np.array(self.Data[self.target])
        print("y", type(y), y.shape)
        y = label_binarize(y, classes = ["No", "Yes"])
        y = np.reshape(y, len(self.Data[self.target]))
        print("After labelization")
        print("y", type(y), y.shape)
        print("y", y)
        X_train, X_test, y_train, y_test = train_test_split(self.Data[self.descriptors], y, test_size = self.fraction,random_state=1992)
        model = SVC(kernel = kernel, probability=True, random_state=1992)
        print(model)
        model.fit(X_train, y_train)
        self.model = model
        self.X_test = X_test
        self.y_test = y_test
        self.kernel = kernel
                        
    def single_prediction(self, Library, Name, target):
        compound = test_compound(self.Data, Library, Name, self.descriptors)   
        result = test_compound_real_category(self.Data, Name, target)
        print("Evaluation of ", str(Name))
        print("Predicted activity value: ", str(self.model.predict(compound)))
        print("Real activity value", result)
    
    def report(self, ref_output):
        Data = self.Data
        predictions = self.model.predict(self.X_test)
        y_score = self.model.decision_function(self.X_test)
        roc_auc, roc_auc_micro = plot_roc(self.y_test, y_score)
        r = svm_report(ref_output,Data, self.kernel,self.fraction ,self.y_test, predictions, self.descriptors, roc_auc)

In [ ]:
a = SVM("Dataset.csv", "PPI", descriptors, 0.3)

In [ ]:
a.train_model('rbf')

In [ ]:
a.single_prediction("PPI", "1602", "PPI")

In [ ]:
a.report("MyfirstSVM")

In [ ]:
#Identify Numerical Data (Descriptors)
def numerical_descriptors(DataFrame):
    print(DataFrame.select_dtypes(np.number).columns)